In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import cross_val_score, KFold
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from statsmodels.stats.outliers_influence import variance_inflation_factor
from sklearn.metrics import accuracy_score

In [2]:
#Import Data Using Pandas Function
# Load the Dataset

url = "https://raw.githubusercontent.com/ankitmandhare1111/Wine-quality-project-ML/refs/heads/main/winequality.csv"
df=pd.read_csv(url)
df.sample(frac=1)


,type,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
2520,white,7.5,0.410,0.23,14.8,0.054,28.0,174.0,0.99898,3.18,0.49,9.7,5
4485,white,6.7,0.310,0.33,2.0,0.033,12.0,74.0,0.99064,3.29,0.65,12.5,6
229,white,7.1,0.240,0.41,17.8,0.046,39.0,145.0,0.99980,3.32,0.39,8.7,5
2028,white,8.9,0.210,0.34,7.1,0.037,33.0,150.0,0.99620,3.10,0.45,9.7,6
6102,red,7.2,0.360,0.46,2.1,0.074,24.0,44.0,0.99534,3.40,0.85,11.0,7
...,...,...,...,...,...,...,...,...,...,...,...,...,...
4510,white,6.8,0.330,0.30,2.1,0.047,35.0,147.0,0.98860,3.24,0.56,13.4,6
6289,red,8.0,0.640,0.22,2.4,0.094,5.0,33.0,0.99612,3.37,0.58,11.0,5
3904,white,5.0,0.455,0.18,1.9,0.036,33.0,106.0,0.98746,3.21,0.83,14.0,7
814,white,7.9,0.350,0.24,15.6,0.072,44.0,229.0,0.99785,3.03,0.59,10.5,6


In [3]:
# statistical info
df.describe()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
count,6487.000000,6489.000000,6494.000000,6495.000000,6495.000000,6497.000000,6497.000000,6497.000000,6488.000000,6493.000000,6497.000000,6497.000000
mean,7.216579,0.339691,0.318722,5.444326,0.056042,30.525319,115.744574,0.994697,3.218395,0.531215,10.491801,5.818378
std,1.296750,0.164649,0.145265,4.758125,0.035036,17.749400,56.521855,0.002999,0.160748,0.148814,1.192712,0.873255
min,3.800000,0.080000,0.000000,0.600000,0.009000,1.000000,6.000000,0.987110,2.720000,0.220000,8.000000,3.000000
25%,6.400000,0.230000,0.250000,1.800000,0.038000,17.000000,77.000000,0.992340,3.110000,0.430000,9.500000,5.000000
50%,7.000000,0.290000,0.310000,3.000000,0.047000,29.000000,118.000000,0.994890,3.210000,0.510000,10.300000,6.000000
75%,7.700000,0.400000,0.390000,8.100000,0.065000,41.000000,156.000000,0.996990,3.320000,0.600000,11.300000,6.000000
max,15.900000,1.580000,1.660000,65.800000,0.611000,289.000000,440.000000,1.038980,4.010000,2.000000,14.900000,9.000000


In [4]:
df.drop(["type"], axis=1, inplace=True)

In [5]:
# check for null values
df.isnull().sum()

fixed acidity           10
volatile acidity         8
citric acid              3
residual sugar           2
chlorides                2
free sulfur dioxide      0
total sulfur dioxide     0
density                  0
pH                       9
sulphates                4
alcohol                  0
quality                  0
dtype: int64

In [7]:
# Check the target column
print(df['quality'].value_counts())


quality
6    2836
5    2138
7    1079
4     216
8     193
3      30
9       5
Name: count, dtype: int64


In [8]:
# Convert to binary classification: Good (1) if quality >= 6, else Bad (0)
df['quality'] = (df['quality'] >= 6).astype(int)

In [9]:
# Separate features and target
X = df.drop('quality', axis=1)
y = df['quality']

In [10]:
# Standardize the features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

In [11]:
# Create DataFrame for VIF
X_vif_df = pd.DataFrame(X_scaled, columns=X.columns)